In [32]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrices
from linearmodels.panel import PanelOLS
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import urlopen
import json
import plotly.graph_objects as go
import geopandas as gpd
import plotly.express as px
%matplotlib inline
mpl.style.use('seaborn')

In [33]:
# Cleaning and reshaping governance indicators data
WGI = pd.read_csv('/Users/matthewbernstein/Downloads/WGI2.csv')

WGI=WGI.rename(columns={'2004 [YR2004]':2004,'2005 [YR2005]':2005,'2006 [YR2006]':2006,'2007 [YR2007]':2007,'2008 [YR2008]':2008,
                        '2009 [YR2009]':2009,'2010 [YR2010]':2010,'2011 [YR2011]':2011,'2012 [YR2012]':2012,
                        '2013 [YR2013]':2013,'2014 [YR2014]':2014,'2015 [YR2015]':2015,
                        '2016 [YR2016]':2016,'2017 [YR2017]':2017,'2018 [YR2018]':2018})
WGI

,Country Name,Country Code,Series Name,Series Code,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,AFG,Control of Corruption: Estimate,CC.EST,-1.345281,-1.443609,-1.43289,-1.587331,-1.638287,-1.534796,-1.635723,-1.579179,-1.419888,-1.436761,-1.354784,-1.341994,-1.526352,-1.517341,-1.496648
1,Afghanistan,AFG,Government Effectiveness: Estimate,GE.EST,-0.8940507,-1.210829,-1.430343,-1.403352,-1.496089,-1.483899,-1.454683,-1.448488,-1.36014,-1.38493,-1.331396,-1.322968,-1.226363,-1.33461,-1.457285
2,Afghanistan,AFG,Political Stability and Absence of Violence/Te...,PV.EST,-2.296179,-2.070934,-2.221302,-2.410581,-2.690477,-2.712689,-2.579152,-2.50206,-2.418561,-2.519349,-2.411068,-2.571222,-2.671054,-2.800609,-2.74673
3,Afghanistan,AFG,Regulatory Quality: Estimate,RQ.EST,-1.505863,-1.637369,-1.667902,-1.688257,-1.621583,-1.674311,-1.532861,-1.540586,-1.193118,-1.193137,-1.118182,-0.9973711,-1.327688,-1.339227,-1.132634
4,Afghanistan,AFG,Rule of Law: Estimate,RL.EST,-1.697143,-1.658668,-1.863189,-1.829666,-1.86438,-1.847092,-1.845436,-1.896632,-1.643165,-1.597314,-1.445648,-1.50404,-1.49819,-1.569692,-1.668478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287,Data from database: Worldwide Governance Indic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
WGI=WGI.drop(['Series Code'],axis=1)
WGI

,Country Name,Country Code,Series Name,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,AFG,Control of Corruption: Estimate,-1.345281,-1.443609,-1.43289,-1.587331,-1.638287,-1.534796,-1.635723,-1.579179,-1.419888,-1.436761,-1.354784,-1.341994,-1.526352,-1.517341,-1.496648
1,Afghanistan,AFG,Government Effectiveness: Estimate,-0.8940507,-1.210829,-1.430343,-1.403352,-1.496089,-1.483899,-1.454683,-1.448488,-1.36014,-1.38493,-1.331396,-1.322968,-1.226363,-1.33461,-1.457285
2,Afghanistan,AFG,Political Stability and Absence of Violence/Te...,-2.296179,-2.070934,-2.221302,-2.410581,-2.690477,-2.712689,-2.579152,-2.50206,-2.418561,-2.519349,-2.411068,-2.571222,-2.671054,-2.800609,-2.74673
3,Afghanistan,AFG,Regulatory Quality: Estimate,-1.505863,-1.637369,-1.667902,-1.688257,-1.621583,-1.674311,-1.532861,-1.540586,-1.193118,-1.193137,-1.118182,-0.9973711,-1.327688,-1.339227,-1.132634
4,Afghanistan,AFG,Rule of Law: Estimate,-1.697143,-1.658668,-1.863189,-1.829666,-1.86438,-1.847092,-1.845436,-1.896632,-1.643165,-1.597314,-1.445648,-1.50404,-1.49819,-1.569692,-1.668478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287,Data from database: Worldwide Governance Indic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
for i in WGI.columns[3:]:
    WGI[i]=pd.to_numeric(WGI[i],errors='coerce')
WGI

,Country Name,Country Code,Series Name,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,AFG,Control of Corruption: Estimate,-1.345281,-1.443609,-1.432890,-1.587331,-1.638287,-1.534796,-1.635723,-1.579179,-1.419888,-1.436761,-1.354784,-1.341994,-1.526352,-1.517341,-1.496648
1,Afghanistan,AFG,Government Effectiveness: Estimate,-0.894051,-1.210829,-1.430343,-1.403352,-1.496089,-1.483899,-1.454683,-1.448488,-1.360140,-1.384930,-1.331396,-1.322968,-1.226363,-1.334610,-1.457285
2,Afghanistan,AFG,Political Stability and Absence of Violence/Te...,-2.296179,-2.070934,-2.221302,-2.410581,-2.690477,-2.712689,-2.579152,-2.502060,-2.418561,-2.519349,-2.411068,-2.571222,-2.671054,-2.800609,-2.746730
3,Afghanistan,AFG,Regulatory Quality: Estimate,-1.505863,-1.637369,-1.667902,-1.688257,-1.621583,-1.674311,-1.532861,-1.540586,-1.193118,-1.193137,-1.118182,-0.997371,-1.327688,-1.339227,-1.132634
4,Afghanistan,AFG,Rule of Law: Estimate,-1.697143,-1.658668,-1.863189,-1.829666,-1.864380,-1.847092,-1.845436,-1.896632,-1.643165,-1.597314,-1.445648,-1.504040,-1.498190,-1.569692,-1.668478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1287,Data from database: Worldwide Governance Indic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
WGI=WGI.set_index(['Country Name','Series Name','Country Code'])
WGI.columns.rename('year',inplace=True)
WGI=WGI.stack().swaplevel(1,3).unstack()
WGI = WGI.reset_index()
WGI

Series Name,Country Name,year,Country Code,Control of Corruption: Estimate,Government Effectiveness: Estimate,Political Stability and Absence of Violence/Terrorism: Estimate,Regulatory Quality: Estimate,Rule of Law: Estimate,Voice and Accountability: Estimate
0,Afghanistan,2004,AFG,-1.345281,-0.894051,-2.296179,-1.505863,-1.697143,-1.203034
1,Afghanistan,2005,AFG,-1.443609,-1.210829,-2.070934,-1.637369,-1.658668,-1.125430
2,Afghanistan,2006,AFG,-1.432890,-1.430343,-2.221302,-1.667902,-1.863189,-1.110294
3,Afghanistan,2007,AFG,-1.587331,-1.403352,-2.410581,-1.688257,-1.829666,-1.057654
4,Afghanistan,2008,AFG,-1.638287,-1.496089,-2.690477,-1.621583,-1.864380,-1.168902
...,...,...,...,...,...,...,...,...,...
3166,Zimbabwe,2014,ZWE,-1.391291,-1.210598,-0.713570,-1.903126,-1.431254,-1.256964
3167,Zimbabwe,2015,ZWE,-1.310461,-1.157869,-0.618156,-1.653554,-1.317532,-1.165904
3168,Zimbabwe,2016,ZWE,-1.259387,-1.158350,-0.619938,-1.720110,-1.368230,-1.180987
3169,Zimbabwe,2017,ZWE,-1.273584,-1.188722,-0.708677,-1.563330,-1.377855,-1.196420


In [37]:
WGI.max()

Series Name
Country Name                                                       Zimbabwe
year                                                                   2018
Country Code                                                            ZWE
Control of Corruption: Estimate                                     2.46999
Government Effectiveness: Estimate                                  2.43697
Political Stability and Absence of Violence/Terrorism: Estimate     1.96506
Regulatory Quality: Estimate                                        2.26054
Rule of Law: Estimate                                               2.10027
Voice and Accountability: Estimate                                  1.80099
dtype: object

In [39]:
WGI.min()

Series Name
Country Name                                                       Afghanistan
year                                                                      2004
Country Code                                                               ABW
Control of Corruption: Estimate                                       -1.86871
Government Effectiveness: Estimate                                    -2.48366
Political Stability and Absence of Violence/Terrorism: Estimate       -3.31494
Regulatory Quality: Estimate                                          -2.64504
Rule of Law: Estimate                                                 -2.60644
Voice and Accountability: Estimate                                    -2.31339
dtype: object

In [45]:
df = px.data.gapminder()
fig = px.choropleth(WGI,locations='Country Code',hover_name='Country Name',color='Control of Corruption: Estimate',animation_frame='year',range_color=[-2,2.25])
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 800
fig

In [46]:
df = px.data.gapminder()
fig = px.choropleth(WGI,locations='Country Code',hover_name='Country Name',color='Government Effectiveness: Estimate',animation_frame='year',range_color=[-2.5,2.5])
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 800
fig

In [47]:
df = px.data.gapminder()
fig = px.choropleth(WGI,locations='Country Code',hover_name='Country Name',color='Government Effectiveness: Estimate',animation_frame='year',range_color=[-3.3,2])
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 800
fig

In [52]:
df = px.data.gapminder()
fig = px.choropleth(WGI,locations='Country Code',hover_name='Country Name',color='Regulatory Quality: Estimate',animation_frame='year',range_color=[-2.6,2.3])
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 800
fig

In [51]:
df = px.data.gapminder()
fig = px.choropleth(WGI,locations='Country Code',hover_name='Country Name',color='Rule of Law: Estimate',animation_frame='year',range_color=[-2.6,2.1])
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 800
fig

In [53]:
df = px.data.gapminder()
fig = px.choropleth(WGI,locations='Country Code',hover_name='Country Name',color='Voice and Accountability: Estimate',animation_frame='year',range_color=[-2.3,1.7])
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 800
fig